## XGB算法梳理

1. 算法原理
1. 损失函数
1. 分裂结点算法
1. 正则化
1. 对缺失值处理
1. 优缺点
1. 应用场景
1. sklearn参数

In [1]:
string = """
小神仙永远开心!
啦啦啦~~"""
string2 = "小神仙"
print("%s %%" %(string))


小神仙永远开心!
啦啦啦~~ %


In [2]:
comp = 3+5j
print(comp)

(3+5j)


In [7]:
a = "1"
A = "2"
print(A)

2


In [38]:
try: 
    a = int(input())
except Exception:                               
    print("请输入正确的数字！")
else:
    print(a)

a
请输入正确的数字！


### 1.
XGBoost算法的基本思想与GBDT类似，不断地地进行特征分裂来生长一棵树，每一轮学习一棵树，其实就是去拟合上一轮模型的预测值与实际值之间的残差。当我们训练完成得到k棵树时，我们要预测一个样本的分数，其实就是根据这个样本的特征，在每棵树中会落到对应的一个叶子节点，每个叶子节点就对应一个分数，最后只需将每棵树对应的分数加起来就是该样本的预测值。

### 2.
如下：
![](https://img-blog.csdnimg.cn/20181224084848792.png)
第一项为训练误差，第二项为正则化项，用来刻画模型的复杂程度。具体形式将在（5.正则化）中介绍。

XGBoost度损失函数进行了二阶泰勒展开：
![](https://img-blog.csdnimg.cn/20181224102938130.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L20wXzM4MDE5ODQx,size_16,color_FFFFFF,t_70)
从损失函数中我们能够看出，我们希望得到一个预测性好的简单模型。预测性好的模型偏差较小，简单模型方差较小。

### 3.
对于每一个叶子节点，尝试进行分裂，分裂前后目标函数的差值为：
![](https://img-blog.csdnimg.cn/20181224090456775.png)
第一项为左子节点的得分，第二项为右子节点的得分，第三项为增加一个额外节点复杂度的损失。

采用**贪心算法**寻找最优分裂点：对每个特征，对它的可能取值进行排序，并通过线性扫描从中找到该特征的最优分裂点，之后找到所有特征中的最优分裂点。
论文中的方法：
- 贪心算法
![](https://img-blog.csdnimg.cn/20181224105917599.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L20wXzM4MDE5ODQx,size_16,color_FFFFFF,t_70)
- 近似算法：针对数据太大，不能直接进行计算
![](https://img-blog.csdnimg.cn/20181224105936349.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L20wXzM4MDE5ODQx,size_16,color_FFFFFF,t_70)

通过特征的分布，按照分布式加权直方图算法确定一组候选分裂点，通过遍历所有的候选分裂点来找到最佳分裂点。 
在寻找split point的时候，不会枚举所有的特征值，而会对特征值进行聚合统计，然后形成若干个bucket(桶)，只将bucket边界上的特征值作为split point的候选，从而获得性能提升。


### 4.
定义了一种模型复杂度（不唯一）：
![](https://img-blog.csdnimg.cn/20181224090035881.png)
T表示叶子节点的数量，第二项表示叶子节点得分的L2范数。

这里出现了γ和λ，这是xgboost自己定义的，在使用xgboost时，可以设定它们的值，显然，γ越大，表示越希望获得结构简单的树，因为此时对较多叶子节点的树的惩罚越大。λ越大也是越希望获得结构简单的树。


### 5.
XGBoost在选择最优分裂点的时候，不考虑该特征的缺失样本。但在后面对样本划分中，会分别将该特征的缺失样本放到左、右节点中，分别计算Gain值，哪边大就把该样本划分到哪一边。如果在训练集中，该特征没有出现缺失样本，但在预测的时候出现缺失样本了，则默认将该样本划分到右节点中。具体方法：
![](https://img-blog.csdn.net/20180320001356662?watermark/2/text/Ly9ibG9nLmNzZG4ubmV0L3FkYnN6c2o=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70)



### 6.
1. XGBoost与GBDT相比，其优势：

 - 将树模型的复杂度加入到正则项中，来避免过拟合，因此泛化性能会优于GBDT。
 - 损失函数用泰勒展开式展开，同时用到了一阶和二阶导数，可以加快优化速度。
 - GBDT只支持CART作为基学习器，XGBoost还支持线性分类器作为基学习器。
 - 引进了特征子采样，像随机森林那样，既能避免过拟合，又能减少计算。
 - 在寻找最优分割点时，考虑到传统的贪心算法效率较低，实现了一种近似贪心算法，用来加速和减少内存小号，除此之外，还考虑了稀疏数据集合缺失值的处理。
 - XGBoost支持并行处理。XGBoost的并行不是模型生成的并行，而是在特征上的并行，将特征排序后以block的形式存储在内存中，在后面迭代重复使用这个结构。这个block也使得并行化成为了可能，其次在节点分裂时，计算每个特征的增益，最终选择增益最大的那个特征去做分割，那么各个特征的增益计算就可以开多线程进行。

2. 与lightGBM相比的不足点：

 - XGBoosting采用预排序，在迭代之前，对结点的特征做预排序，遍历选择最优分割点，数据量大时，贪心法耗时，LightGBM方法采用histogram算法，占用的内存低，数据分割的复杂度更低。
 - XGBoosting采用level-wise生成决策树，同时分裂同一层的叶子，从而进行多线程优化，不容易过拟合，但很多叶子节点的分裂增益较低，没必要进行跟进一步的分裂，这就带来了不必要的开销；LightGBM采用深度优化，leaf-wise生长策略，每次从当前叶子中选择增益最大的结点进行分裂，循环迭代，但会生长出更深的决策树，产生过拟合，因此引入了一个阈值进行限制，防止过拟合。

### 7.
大规模机器学习

### 8.
[我拒绝回答这个问题](https://xgboost.apachecn.org/#/docs/15)